In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from researchlib.single_import import *

In [3]:
train_loader = FromTextDataset(text.AGNews(True), 32)
test_loader = FromTextDataset(text.AGNews(False), 32)

VOC: 79445
VOC: 23139


In [60]:
model = builder([
    nn.Embedding(79445, 512),
    Permute((0,2,1)),
    PositionEncoding(512),
    MultiHeadAttention(512, 512),
#     MultiHeadAttention(512, 512),
#     MultiHeadAttention(512, 512),
#     LSTMLayer(512, 512, return_sequences=True),
    TimeDistributed(nn.Linear(512, 1)),
    TimeDistributed(nn.ReLU()),
    Flatten(),
    nn.Linear(60, 4),
    nn.LogSoftmax(-1)
])

In [61]:
runner = Runner(model, test_loader, test_loader, 'adam', 'nll')

In [62]:
runner.fit(10, 3e-5)

  Epochs    train_loss    train_acc      val_loss      val_acc    
    1*        1.3891        0.2517        1.3889        0.2500    


    2*        1.3887        0.2500        1.3886        0.2500    


    3*        1.3885        0.2500        1.3884        0.2500    


    4*        1.3882        0.2500        1.3882        0.2500    


KeyboardInterrupt: 

In [ ]:
x = np.load('0419_1_x_data.npy').transpose(0,2,1)
y = np.load('0419_1_y_data.npy')[:, None, :]

x, y = shuffle((x, y))

In [ ]:
train_x, test_x = x[:int(len(x)*0.7)], x[int(len(x)*0.7):]
train_y, test_y = y[:int(len(y)*0.7)], y[int(len(y)*0.7):]

train_dataset = FromNumpy(train_x, train_y, 32)
test_dataset = FromNumpy(test_x, test_y, 32)

In [ ]:
class L(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x, y):
        x = x.view(x.size(0), -1)
        y = y.view(y.size(0), -1)
        return torch.abs(x-y).mean()

In [ ]:
model = builder([
    PositionEncoding(64),
    TimeDistributed(nn.Linear(7, 64)),
    MultiHeadAttention(64, 64),
    MultiHeadAttention(64, 64),
    TimeDistributed(nn.Linear(64, 1)),
])

In [ ]:
runner = Runner(model, train_dataset, test_dataset, 'adam', L())
#runner = Runner(model, train_dataset, test_dataset, 'adam', {'mse':5, 'mae':10})  # loss ensemble!

In [ ]:
runner.fit(30, 1e-4)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x, y = next(iter(train_dataset))
model = runner.model.eval()
r = model(x.cuda())
plt.plot(r[0][0].detach().cpu().numpy())
plt.plot(y[0][0].numpy())
plt.show()

In [ ]:
import pandas as pd
import numpy as np

data_name = 'test.csv'
df = pd.read_csv(data_name)

x_data = np.stack((df['ta_red_cards_1'].values,
                   df['ta_red_cards_2'].values,
                   df['ta_ft_ht_hdp_sp'].values,
                   df['ta_ft_ht_hdp_x_n1'].values,
                   df['ta_ft_ht_hdp_x_0'].values,
                   df['ta_ft_ht_hdp_x_1'].values,
                   (df['ta_match_time']/5400.0).values
                  ), axis=1)

print(x_data.shape)
df.head()

In [ ]:
import numpy as np
import random

forecast_n1 = []
forecast_0 = []
forecast_1 = []

rc1 = []
rc2 = []

for i in range(len(df)):
    if i < 1:
        forecast_n1.append(0)
        forecast_0.append(0)
        forecast_1.append(0)
        
        rc1.append(0)
        rc2.append(0)
    else:
        forecast_n1.append(df['ta_ft_ht_hdp_x_n1'][i]-df['ta_ft_ht_hdp_x_n1'][i-1])
        forecast_0.append(df['ta_ft_ht_hdp_x_0'][i]-df['ta_ft_ht_hdp_x_0'][i-1])
        forecast_1.append(df['ta_ft_ht_hdp_x_1'][i]-df['ta_ft_ht_hdp_x_1'][i-1])

        rc1.append(df['ta_red_cards_1'][i]-df['ta_red_cards_1'][i-1])
        rc2.append(df['ta_red_cards_2'][i]-df['ta_red_cards_2'][i-1])
        
for i in range(len(df)): 

    x_data[i, -4] = forecast_n1[i]
    x_data[i, -3] = forecast_0[i]
    x_data[i, -2] = forecast_1[i]
    
    x_data[i, 0] = rc1[i]
    x_data[i, 1] = rc2[i]

    if pd.isnull(x_data).any():
        x_data[i] = x_data[i-1]

print(x_data.shape)

In [ ]:
y_predict = []
model = runner.model.eval()

for i in range(1, len(x_data)):
    if i < 2:
        r = model(torch.from_numpy(np.array(x_data)[None, :i, :].transpose(0, 2, 1)).cuda().float())
        y_predict.append(r[0,0,-1].detach().cpu().numpy())
    else:
        if i < 600:
            first = np.array(x_data)[None, 0:i:2, :]
            second = np.array(x_data)[None, 1:i:2, :]
            first = first[:, :second.shape[1], :]
            inp = (first+second)/2
            inp = torch.from_numpy(inp.transpose(0, 2, 1)).cuda().float()
            r = model(inp)
            y_predict.append(r[0,0,-1].detach().cpu().numpy())
        else:
            inp = (np.array(x_data)[None, i-600:i:2,:]+ \
                                                    np.array(x_data)[None, i-600+1:i:2, :])/2.0
            inp = torch.from_numpy(inp.transpose(0, 2, 1)).cuda().float()
            r = model(inp)
            y_predict.append(r[0,0,-1].detach().cpu().numpy())


In [ ]:
y_predict = np.array(y_predict)

In [ ]:
pd.DataFrame(y_predict).plot(style='.', markersize=1, color='magenta', figsize=(10, 10))
df['OP_HK'].plot(style='.', markersize=1, color='orange')